In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import logging
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from PipelineCacheWrapper.PipelineCacheWrapper import PipelineCacheWrapper

In [2]:
from LamaTRExData import LamaTRExData
from SentenceTypologyQueryResults import SentenceTypologyQueryResults
from relation_templates.templates import relations, nominalized_relations

In [3]:
from TypologyQuerier import TypologyQuerier 

In [4]:
logging.set_verbosity_error()

In [5]:
MASK = "[MASK]"
KEYS = ["active", "passive", "nominalized"]

In [6]:
#relations = ["P19", "P36", "P101", "P103","P106","P108", "P178", "P1001"]
#relations = ["P19", "P413", "P159", "P103"]
#relations = ["P364"]
#relations = nominalized_relations

In [7]:
TOP_K = 1

In [8]:
#unmasker = pipeline('fill-mask', model='bert-large-uncased-whole-word-masking', top_k=100)
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=TOP_K)

In [9]:
TREx = LamaTRExData(relations = relations)
TREx.load()

In [10]:
querier = TypologyQuerier(unmasker, relations, TOP_K, MASK)

In [11]:
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 33561.27it/s]


In [12]:
%%time
unmasker.save_to_cache()

CPU times: user 99.7 ms, sys: 15.2 ms, total: 115 ms
Wall time: 113 ms


In [13]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      9594 |   34039 |    28.1853 |
| compound         |      4452 |   34039 |    13.0791 |
| complex          |      4421 |   34039 |    12.988  |
| compound-complex |      5413 |   34039 |    15.9023 |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       610 |     937 |    65.1014 |
| compound         |       604 |     937 |    64.461  |
| complex          |       595 |     937 |    63.5005 |
| compound-complex |       451 |     937 |    48.1323 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      6689 |   20928 |    31.962  |
| compound         |      3040 |   20928 |    14.526  |
| complex          |      3487 |   20928 |    16.

In [14]:
querier.print_result()

P159: headquarters location N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       313 |     967 |    32.3681 |
| compound         |       334 |     967 |    34.5398 |
| complex          |       329 |     967 |    34.0228 |
| compound-complex |       330 |     967 |    34.1262 |
P37: official language N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       527 |     966 |    54.5549 |
| compound         |       391 |     966 |    40.4762 |
| complex          |       258 |     966 |    26.7081 |
| compound-complex |       372 |     966 |    38.5093 |
P1412: languages spoken, written or signed N:M
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       630 |     969 |   65.0155  |
| compound         |        23 |     969 |    2.37358 

In [15]:
querier.print_for_latex()

P1376 & capital of & 1:1 & 73.93 & 72.65 & 73.93 & 73.50 \\
P36 & capital & 1:1 & 62.16 & 61.74 & 60.03 & 39.69 \\
P103 & native language & N:1 & 72.16 & 76.15 & 76.36 & 69.91 \\
P127 & owned by & N:1 & 34.79 & 0.00 & 1.46 & 25.76 \\
P131 & located in the administrative territorial entity & N:1 & 23.27 & 0.00 & 0.00 & 0.11 \\
P136 & genre & N:1 & 0.64 & 0.00 & 4.19 & 0.00 \\
P138 & named after & N:1 & 3.88 & 8.22 & 3.88 & 12.09 \\
P140 & religion or worldview & N:1 & 75.05 & 47.36 & 74.84 & 37.84 \\
P159 & headquarters location & N:1 & 32.37 & 34.54 & 34.02 & 34.13 \\
P17 & country & N:1 & 31.29 & 3.66 & 35.81 & 15.38 \\
P176 & manufacturer & N:1 & 85.64 & 66.50 & 40.02 & 80.86 \\
P19 & place of birth & N:1 & 21.08 & 0.32 & 0.21 & 0.00 \\
P20 & place of death & N:1 & 27.91 & 0.00 & 0.10 & 0.00 \\
P264 & record label & N:1 & 9.32 & 0.23 & 0.00 & 0.00 \\
P276 & location & N:1 & 41.50 & 39.83 & 33.79 & 41.81 \\
P279 & subclass of & N:1 & 30.60 & 3.42 & 9.96 & 2.07 \\
P30 & continent & N:1

In [16]:
TOP_K = 10

In [17]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=TOP_K)

In [18]:
querier.reset()
querier.set_top_k(TOP_K)
querier.set_model(unmasker)

In [19]:
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 28707.69it/s]


In [20]:
%%time
unmasker.save_to_cache()

CPU times: user 920 ms, sys: 73.1 ms, total: 993 ms
Wall time: 992 ms


In [21]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |     19653 |   34039 |    57.7367 |
| compound         |     12777 |   34039 |    37.5364 |
| complex          |     12727 |   34039 |    37.3895 |
| compound-complex |     13329 |   34039 |    39.158  |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       767 |     937 |    81.857  |
| compound         |       762 |     937 |    81.3234 |
| complex          |       757 |     937 |    80.7898 |
| compound-complex |       727 |     937 |    77.588  |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |     13441 |   20928 |    64.225  |
| compound         |      8037 |   20928 |    38.4031 |
| complex          |      9297 |   20928 |    44.

In [22]:
querier.print_result()

P159: headquarters location N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       499 |     967 |    51.6029 |
| compound         |       569 |     967 |    58.8418 |
| complex          |       557 |     967 |    57.6008 |
| compound-complex |       550 |     967 |    56.8769 |
P37: official language N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       879 |     966 |    90.9938 |
| compound         |       841 |     966 |    87.06   |
| complex          |       823 |     966 |    85.1967 |
| compound-complex |       807 |     966 |    83.5404 |
P1412: languages spoken, written or signed N:M
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       872 |     969 |    89.9897 |
| compound         |       837 |     969 |    86.3777 

In [23]:
querier.print_for_latex()

P1376 & capital of & 1:1 & 91.45 & 91.45 & 91.03 & 91.03 \\
P36 & capital & 1:1 & 78.66 & 77.95 & 77.38 & 73.12 \\
P103 & native language & N:1 & 96.42 & 97.85 & 97.34 & 97.54 \\
P127 & owned by & N:1 & 61.86 & 12.52 & 18.92 & 54.88 \\
P131 & located in the administrative territorial entity & N:1 & 54.48 & 16.12 & 21.57 & 10.90 \\
P136 & genre & N:1 & 44.15 & 2.58 & 49.52 & 1.50 \\
P138 & named after & N:1 & 69.46 & 64.34 & 22.95 & 68.37 \\
P140 & religion or worldview & N:1 & 98.73 & 97.04 & 97.46 & 86.05 \\
P159 & headquarters location & N:1 & 51.60 & 58.84 & 57.60 & 56.88 \\
P17 & country & N:1 & 67.53 & 52.69 & 73.55 & 75.38 \\
P176 & manufacturer & N:1 & 94.30 & 90.33 & 70.67 & 90.84 \\
P19 & place of birth & N:1 & 47.67 & 3.60 & 2.22 & 2.44 \\
P20 & place of death & N:1 & 60.86 & 3.15 & 1.78 & 1.57 \\
P264 & record label & N:1 & 31.47 & 20.98 & 2.33 & 20.98 \\
P276 & location & N:1 & 65.59 & 61.52 & 59.54 & 63.30 \\
P279 & subclass of & N:1 & 68.98 & 16.80 & 22.93 & 16.29 \\
P30 

In [24]:
TOP_K = 100

In [25]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=TOP_K)

EOFError: Ran out of input

In [ ]:
querier.reset()
querier.set_top_k(TOP_K)
querier.set_model(unmasker)

In [ ]:
querier.query(TREx.data)

In [ ]:
%%time
unmasker.save_to_cache()

In [ ]:
querier.print_global_result()

In [ ]:
querier.print_result()

In [ ]:
querier.print_for_latex()